In [1]:
import math
import requests

import ipywidgets as widgets
from ipywidgets import interactive_output, HBox, Layout, interact_manual
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

In [2]:
pio.renderers.default = "iframe"

In [3]:
def read_latest_ecdc_file(file_link=None, file_date=None, max_consecutive_dates=5, walk_back=True):
    """
    Returns a DataFrame of the latest Covid-19 numbers by country as posted daily on the ECDC website.
    
    Args:
        file_link (str): Download link for daily .xls file. If file_link is None, the function defaults to the last known location as a convenience.
        file_date (Union[str, datetime.date, pd.Timestamp]): Starting date for file download tries. If file_date is None, the function defaults to today.
        max_consecutive_dates (int): Number of days to walk back or forward if the current date's file is not available.
        walk_back (bool): Way to increment dates for file matching in time. If walk_back is True, the function will decrement dates until max_consecutive_dates is reached or a valid link is found. 
        Otherwise, the function will increment dates.
        
    Returns:
        pd.DataFrame
    """    
    if file_link is None:
        file_link = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"
    
    if file_date is None:
        file_date = pd.Timestamp.today()
    else:
        file_date = pd.Timestamp(file_date)
    
    try:
        latest_link = file_link + "{:%Y-%m-%d}.xlsx".format(file_date)
        resp = requests.get(latest_link)
    except:
        print("Invalid URL.")
        return

    while resp.status_code != 200 and max_consecutive_dates > 1:
        print("File retrieval failed for {:%Y-%m-%d}".format(file_date))
        if walk_back:
            file_date -= pd.Timedelta("1d")
        else:
            file_date += pd.Timedelta("1d")
        max_consecutive_dates -= 1
        latest_link = file_link + "{:%Y-%m-%d}.xlsx".format(file_date)
        resp = requests.get(latest_link)
    
    if resp.status_code != 200:
        print("File retrieval failed for {:%Y-%m-%d}.".format(file_date))
        print("Maximum number of consecutive dates reached. Please check if URL is correct or expand date window using max_consecutive_dates argument.")
        return
    
    print("Latest file date: {:%Y-%m-%d}.".format(file_date))
    
    df = pd.read_excel(resp.content)
    df.columns = ["date_rep", "day", "month", "year", "cases", "deaths", "country", "alpha_2_code", "alpha_3_code", "population_2018"]
    
    return df

In [4]:
df_ecdc = read_latest_ecdc_file()

Latest file date: 2020-04-09.


In [5]:
def read_wiki_iso_country_codes(page_link=None, table_index=0, matching_length=True):
    """
    Returns a DataFrame of country codes from Wikipedia.
    
    Args:
        page_link (str): Link to the Wikipedia page of ISO country codes.
        table_index (int): Index of the table within the HTML elements.
        matching_length (bool): If True, only keeps ISO codes that respect string length requirements for both columns.
        
    Returns:
        pd.DataFrame
    """
    # Get HTML tables
    if page_link is None:
        tables = pd.read_html(r"https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes")
    else:
        tables = pd.read_html(page_link)
    
    # Load & format
    df = tables[table_index]
    df.columns = df.columns.droplevel()
    df.columns = [c.split("[")[0].strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
    df["alpha_2_code"].iloc[0] = "AF"
    
    # Filter out countries that have alpha_3_code longer than 3 characters
    if matching_length:
        len_before = df.shape[0]
        df = df[df["alpha_3_code"].str.len() == 3].copy()
        len_after = df.shape[0]
        print("{:,.0f} entries dropped from 3-letter ISO codes.".format(len_before - len_after))
        
    return df

In [6]:
df_wiki = read_wiki_iso_country_codes()

31 entries dropped from 3-letter ISO codes.


In [7]:
def read_regions_continents_table(page_link=None, table_index=2):
    """
    Returns a DataFrame of region and continent classification for each country.
    
    Args:
        page_link (str): Link to the webpage.
        table_index (int): Index of the table within the HTML elements.
        
    Returns:
        pd.DataFrame
    """
    # Get HMTL tables
    if page_link is None:
        tables = pd.read_html(r"http://statisticstimes.com/geography/countries-by-continents.php")
    else:
        tables = pd.read_html(page_link)
    
    # Load & format
    df = tables[table_index]
    df.drop(columns=["No"], inplace=True)
    df.columns = [c.strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
    
    return df

In [8]:
df_geo_agg = read_regions_continents_table()

In [9]:
def create_full_dataset(df_ecdc=None, df_wiki=None, df_geo_agg=None):
    """
    Returns a clean dataset composed by assembling ECDC data and Wikipedia ISO country data. Ensures all countries are included and span all dates for compatibility with Plotly. 
    Adds additional metrics per country: cumulative cases, cumulative deaths, mortality rate, % of population infected, % of population deaths.
    
    Args:
        df_ecdc (pd.DataFrame): DataFrame from raw data published daily by the ECDC.
        df_wiki (pd.DataFrame): DataFrame of table of country codes from Wikipedia.
        df_geo_agg (pd.DataFrame): DataFrame of regional and continental classification for each country.
        
    Returns:
        pd.DataFrame
    """
    # Safety check
    if any(x is None for x in [df_ecdc, df_wiki, df_geo_agg]):
        print("One or more DataFrame is missing from the arguments, please check.")
        return
    
    # Format return DataFrame
    df = df_ecdc.copy()
    df.drop(columns=["day", "month", "year"], inplace=True)
    df["date_rep"] = df["date_rep"].astype(str)
    df["cum_cases"] = None
    df["cum_deaths"] = None
    df.dropna(subset=["alpha_3_code"], inplace=True)
    
    # Get list of all dates covered and missing countries
    all_dates = pd.date_range(df["date_rep"].min(), df["date_rep"].max()).astype(str).tolist()
    missing_countries = list(set(df_wiki["alpha_3_code"]).difference(set(df_ecdc["alpha_3_code"])))
    frames = []
    
    # Fill in dates for countries included in the ECDC DataFrame
    for alpha_3_code in df["alpha_3_code"].unique():
        try:
            df_country = df[df["alpha_3_code"] == alpha_3_code].copy()
            # Additional dates
            dates_to_add = list(set(all_dates).difference(set(df_country["date_rep"])))
            df_to_add = pd.DataFrame({"date_rep": dates_to_add})
            df_to_add["cases"] = 0
            df_to_add["deaths"] = 0
            df_to_add["country"] = df_country["country"].iloc[0]
            df_to_add["alpha_2_code"] = df_country["alpha_2_code"].iloc[0]
            df_to_add["alpha_3_code"] = alpha_3_code
            df_to_add["population_2018"] = df_country["population_2018"].iloc[0]
            # Concatenate both
            df_temp = pd.concat([df_country, df_to_add], ignore_index=True)
            df_temp.sort_values(by="date_rep", inplace=True)
            # Add cumulative counts
            df_temp["cum_cases"] = df_temp["cases"].cumsum()
            df_temp["cum_deaths"] = df_temp["deaths"].cumsum()
            frames.append(df_temp)
        except:
            print("Issue encountered while adding dates to country code {}".format(alpha_3_code))
        
    # Fill in missing countries
    for alpha_3_code in missing_countries:
        df_missing = pd.DataFrame({"date_rep": all_dates})
        df_missing["cases"] = 0
        df_missing["deaths"] = 0
        df_missing["country"] = df_wiki.loc[df_wiki["alpha_3_code"] == alpha_3_code, "country_name"].iloc[0]
        df_missing["alpha_2_code"] = df_wiki.loc[df_wiki["alpha_3_code"] == alpha_3_code, "alpha_2_code"].iloc[0]
        df_missing["alpha_3_code"] = alpha_3_code
        df_missing["population_2018"] = np.NaN
        df_missing["cum_cases"] = 0
        df_missing["cum_deaths"] = 0
        frames.append(df_missing)
    
    # Create full DataFrame and order by date
    df = pd.concat(frames, ignore_index=True)
    df.sort_values(by=["country", "date_rep"], inplace=True)
    # Add indicators
    df["mortality_rate"] = (df["cum_deaths"] / df["cum_cases"]).fillna(0)
    df["fraction_infected"] = df["cum_cases"] / df["population_2018"]
    df["fraction_deaths"] = df["cum_deaths"] / df["population_2018"]
    df["infections_growth_rate"] = df["cum_cases"].pct_change()
    df.loc[df["date_rep"] == df["date_rep"].min(), "infections_growth_rate"] = np.NaN
    df.loc[df["infections_growth_rate"] == math.inf, "infections_growth_rate"] = 1
    df["deaths_growth_rate"] = df["cum_deaths"].pct_change()
    df.loc[df["date_rep"] == df["date_rep"].min(), "deaths_growth_rate"] = np.NaN
    df.loc[df["deaths_growth_rate"] == math.inf, "deaths_growth_rate"] = 1
    # Merge region and continent classification
    df = df.merge(df_geo_agg[["iso_alpha3_code", "region_1", "region_2", "continent"]], how="left", left_on="alpha_3_code", right_on="iso_alpha3_code").drop(columns=["iso_alpha3_code"])
    # Clean
    df.loc[df["country"] == "Kosovo", "continent"] = "Europe"
    df.loc[df["country"] == "Taiwan", "continent"] = "Asia"
    df.loc[df["country"] == "Bonaire", "continent"] = "South America"
    df["country"] = df["country"].str.replace("_", " ")
    
    return df

In [10]:
df_clean = create_full_dataset(df_ecdc, df_wiki, df_geo_agg)

In [11]:
def generate_animated_map(df=df_clean.copy(), scope="world", metric="cum_cases", chart_type="choropleth", fit_dates=True):
    """
    Generates an animated map from ECDC Covid-19 country-level data.
    
    Args:
        df (pd.DataFrame): data to be visualized.
        scope (str): geographical scope of the map. Can be one of 'world', 'europe', 'asia', 'africa', 'north america', 'south america'. Defaults to 'world'.
        metric (str): metric to plot. Can be one of 'cases', 'deaths', 'cum_cases', 'cum_deaths', 'mortality_rate', 'fraction_infected', 'fraction_deaths', 'infections_growth_rate', 'deaths_growth_rate'. Defaults to 'cum_cases'.
        chart_type (str): type of map. Can be one of 'choropleth' or 'scatter_geo'.
        fit_dates (bool): drop dates before data starts being non-zero for selected criteria.
        
    Returns:
        None
    """
    # Title and legend dict
    title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
        
    # Filter df to match scope
    if scope.lower() != "world":
        df = df[df["continent"] == scope.title()].copy()
        
    if scope not in ["world", "europe", "asia", "africa", "north america", "south america"]:
        print("Value of scope not recognized ({}). Check function signature for supported values".format(scope))
        return
    
    # Fit dates to selection
    if fit_dates:
        min_date = df.loc[df[metric] > 0, "date_rep"].min()
        df = df[df["date_rep"] >= min_date].copy()
    
    # Generate figure
    if chart_type == "choropleth":
        fig = px.choropleth(df, locations="alpha_3_code", color=metric, hover_name="country", hover_data=["date_rep", metric], range_color=[0, df[metric].max()],
                           color_continuous_scale=px.colors.sequential.Reds, scope=scope, animation_frame="date_rep")
        fig.update_layout(coloraxis_colorbar=dict(title=title_legend_dict[metric].capitalize(), thicknessmode="pixels", thickness=25, lenmode="pixels", len=397, yanchor="middle", y=.5, ticks="outside"))
    else:
        fig = px.scatter_geo(df, locations="alpha_3_code", hover_name="country", size=metric, size_max=60, color_discrete_sequence=[px.colors.sequential.Reds[-2]], scope=scope, animation_frame="date_rep")
    
    # Format figure
    fig.update_geos(showcountries=True, countrycolor="black")
    fig.update_layout(margin=dict(r=0, t=0, l=0, b=0), width=900, height=700, title_text="<b>Covid-19 {} by country</b>".format(title_legend_dict[metric]), title_y=.99, title_yanchor="top", title_x=0.5, title_xanchor="center")
    fig.layout.template = "plotly_dark"
    pio.show(fig)

In [12]:
# Controls
map_scope_list = sorted([("World", "world")] + [(c, c.lower()) for c in df_clean["continent"].unique() if c not in ["Antarctica", "Oceania"]])
map_scope_dropdown = widgets.Dropdown(options=map_scope_list, value="world", description="Scope")

map_title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
map_metric_list = [(v.capitalize(), k) for k, v in map_title_legend_dict.items()]
map_metric_dropdown = widgets.Dropdown(options=map_metric_list, value="cum_cases", description="Metric")

map_chart_type_list = [("Heatmap", "choropleth"), ("Scatter map", "scatter_geo")]
map_chart_dropdown = widgets.Dropdown(options=map_chart_type_list, value="choropleth", description="Chart type")

# Layout & display
map_box_layout = Layout(justify_content="flex-start", align_items="center")
map_ui = HBox([map_scope_dropdown, map_metric_dropdown, map_chart_dropdown], layout=map_box_layout)
map_out = interactive_output(generate_animated_map, dict(scope=map_scope_dropdown, metric=map_metric_dropdown, chart_type=map_chart_dropdown))

In [13]:
# Scaling utility functions
def scale_upper_limit(x):
    sign = np.sign(x)
    x = abs(x)
    if x == 0:
        return 0
    elif x >= 1:
        power = int(math.log10(x))
        scale = math.pow(10, power)
        lower = math.floor(x / scale) * scale
        for step in [1.25, 1.5, 1.75, 2]:
            if x < lower * step:
                return lower * step * sign
    else:
        factor = 1
        while x < 1:
            factor *= 10
            x *= 10
        return scale_upper_limit(x * sign) / factor
    
    
def scale_lower_limit(x):
    sign = np.sign(x)
    x = abs(x)
    if x == 0:
        return 0
    elif x >= 1:
        power = int(math.log10(x))
        scale = math.pow(10, power)
        lower = math.floor(x / scale) * scale
        for step in [1.75, 1.5, 1.25, 1]:
            if x >= lower * step:
                return lower * step * sign
    else:
        factor = 1
        while x < 1:
            factor *= 10
            x *= 10
        return scale_lower_limit(x * sign) / factor
    
    
def generate_scale(values):
    if all(x < 0 for x in values):
        return scale_upper_limit(min(values)), scale_lower_limit(max(values))
    else:
        return scale_lower_limit(min(values)), scale_upper_limit(max(values))

In [14]:
def generate_animated_bar_chart(df=df_clean.copy(), scope="world", x="cum_cases", y="default", x_cutoff=None, top_n=None, fit_dates=True):
    """
    Generates an animated bar chart from ECDC Covid-19 country-level data.
    
    Args:
        df (pd.DataFrame): ECDC data to be visualized.
        scope (str): geographical scope of the data. Can be one of 'world', 'europe', 'asia', 'africa', 'north america', 'south america'. Defaults to 'world'.
        x (str): metric to plot. Can be one of 'cases', 'deaths', 'cum_cases', 'cum_deaths', 'mortality_rate', 'fraction_infected', 'fraction_deaths', 'infections_growth_rate', 'deaths_growth_rate'. Defaults to 'cum_cases'.
        y (str): unit to group data. If scope is 'world', defaults to 'continent', otherwise defaults to 'country'.
        x_cutoff (int): will not represent y groups that fall strictly below x_cutoff value. If x is cumulative, cutoff value applies to the last value by date. If not, cutoff value applies to the maximum within the date range.
        top_n (int): only display top N y groups. Operates like x_cutoff.
        fit_dates (bool): drop dates before data starts being non-zero for selected criteria.
        
    Returns:
        None
    """
    # Title and legend dict
    title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
    # Check
    if scope.lower() not in ["world", "europe", "asia", "africa", "north america", "south america"]:
        print("Value of scope not recognized ({}). Check function signature for supported values.".format(scope))
        return
    
    # Filter df to match scope
    if scope.lower() != "world":
        df = df[df["continent"] == scope.title()].copy()
        
    if y == "default":
        if scope == "world":
            y = "continent"
        else:
            y = "country"
    
    # Metric cutoff/top n aggregates
    if "cum" in x:
        df_check = df.loc[df["date_rep"] == df["date_rep"].max(), [y, x]].groupby(y).sum()
    else:
        df_check = df[[y, x]].groupby(y).agg(max)
            
    if x_cutoff is not None:
        y_to_remove = df_check.loc[df_check[x] < x_cutoff].index
        df = df[~df[y].isin(y_to_remove)].copy()
        
    if top_n is not None:
        if len(df_check.index) > top_n:
            y_to_keep = df_check.nlargest(top_n, x).index
            df = df[df[y].isin(y_to_keep)].copy()
    
    # Fit dates to selection
    if fit_dates:
        min_date = df.loc[df[x] > 0, "date_rep"].min()
        df = df[df["date_rep"] >= min_date].copy()
    
    # Formatting args
    max_val = df_check.max()[x]
    graph_scale = [0, scale_upper_limit(max_val)]
    graph_height = 80 * len(df[y].unique())
    
    # Generate & layout figure
    fig = px.bar(df, x=x, y=y, color=y, orientation="h", hover_name=y, hover_data=["date_rep", "country", y, x], animation_frame="date_rep", range_x=graph_scale, labels={x: "", y: ""}, color_discrete_sequence=px.colors.cyclical.Twilight)
    fig.update_layout(width=1800, height=graph_height, title_text="<b>Covid-19 {} by {}</b>".format(title_legend_dict[x], y), title_y=.99, title_yanchor="top", title_x=0.5, title_xanchor="center")
    fig.layout.template = "plotly_dark"
    pio.show(fig)

In [15]:
# Controls definition
scope_list = sorted([("World", "world")] + [(c, c.lower()) for c in df_clean["continent"].unique() if c not in ["Antarctica", "Oceania"]])
bar_scope_dropdown = widgets.Dropdown(options=scope_list, value="world", description="Scope")

title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
metric_list = [(v.capitalize(), k) for k, v in title_legend_dict.items()]
bar_metric_dropdown = widgets.Dropdown(options=metric_list, value="cum_cases", description="Metric")

bar_cutoff_text = widgets.FloatText(value=0, description="Metric cutoff", disabled=False)
bar_top_n_text = widgets.IntText(value=10, description="Show top", disabled=False)

# Reset cutoff to 0 & top_n to 10 when dropdowns change
def dropdown_value_change(change):
    bar_cutoff_text.value = 0
    bar_top_n_text.value=10
    
bar_scope_dropdown.observe(dropdown_value_change, names="value")
bar_metric_dropdown.observe(dropdown_value_change, names="value")

# Layout & display
bar_box_layout = Layout(justify_content="flex-start", align_items="center")
bar_ui = HBox([bar_scope_dropdown, bar_metric_dropdown, bar_cutoff_text, bar_top_n_text], layout=bar_box_layout)
bar_out = interactive_output(generate_animated_bar_chart, dict(scope=bar_scope_dropdown, x=bar_metric_dropdown, x_cutoff=bar_cutoff_text, top_n=bar_top_n_text))

In [16]:
def generate_animated_scatter_plot(df=df_clean.copy(), x="cum_cases", y="mortality_rate", size="population_2018", facet_col="continent"):
    """
    
    """
    # Title and legend dict
    title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
    
    # Filter dataset for missing metrics
    if df[size].isnull().sum() > 0:
        df = df[df[size].notnull()].copy()
    
    x_upper, y_upper = scale_upper_limit(df[x].max()), scale_upper_limit(df[y].max())
    x_scale = [0 - 0.1 * x_upper, x_upper]
    y_scale = [0 - 0.1 * y_upper, y_upper]
    
    fig = px.scatter(df, x=x, y=y, range_x=x_scale, range_y=y_scale, size=size, facet_col=facet_col, hover_name="country", hover_data=["continent", x, y], size_max=60, animation_frame="date_rep", 
                     animation_group="alpha_3_code", labels={x: title_legend_dict[x].capitalize(), y: title_legend_dict[y].capitalize()}, color="continent")
    fig.update_layout(width=2000, height=600, title_text="<b>Covid-19 {} vs {} by continent & country</b>".format(title_legend_dict[x], title_legend_dict[y]), title_y=.99, title_yanchor="top", title_x=0.5, title_xanchor="center")
    fig.layout.template = "plotly_dark"
    pio.show(fig)

In [17]:
# Controls
title_legend_dict = {"cases": "new cases", "deaths": "new deaths", "cum_cases": "cumulative cases", "cum_deaths": "cumulative deaths", "mortality_rate": "mortality rate", "fraction_infected": "% of pop. infected",
                        "fraction_deaths": "% of pop. dead", "infections_growth_rate": "infections growth rate", "deaths_growth_rate": "deaths growth rate"}
x_list = [(v.capitalize(), k) for k, v in title_legend_dict.items()]
scatter_x_dropdown = widgets.Dropdown(options=x_list, value="cum_cases", description="x")
scatter_y_dropdown = widgets.Dropdown(options=x_list, value="mortality_rate", description="y")
size_list = x_list + [("2018 population", "population_2018")]
scatter_size_dropdown = widgets.Dropdown(options=size_list, value="population_2018", description="Marker size")

# Layout & display
scatter_box_layout = Layout(justify_content="flex-start", align_items="center")
scatter_ui = HBox([scatter_x_dropdown, scatter_y_dropdown, scatter_size_dropdown], layout=scatter_box_layout)
scatter_out = interactive_output(generate_animated_scatter_plot, dict(x=scatter_x_dropdown, y=scatter_y_dropdown, size=scatter_size_dropdown))

In [18]:
display(map_ui, map_out, bar_ui, bar_out, scatter_ui, scatter_out)

Output()

Output()

Output()

In [27]:
scatter_out.observe?

Signature: scatter_out.observe(handler, names=traitlets.All, type='change')
Docstring:
Setup a handler to be called when a trait changes.

This is used to setup dynamic notifications of trait changes.

Parameters
----------
handler : callable
    A callable that is called when a trait changes. Its
    signature should be ``handler(change)``, where ``change`` is a
    dictionary. The change dictionary at least holds a 'type' key.
    * ``type``: the type of notification.
    Other keys may be passed depending on the value of 'type'. In the
    case where type is 'change', we also have the following keys:
    * ``owner`` : the HasTraits instance
    * ``old`` : the old value of the modified trait attribute
    * ``new`` : the new value of the modified trait attribute
    * ``name`` : the name of the modified trait attribute.
names : list, str, All
    If names is All, the handler will apply to all traits.  If a list
    of str, handler will apply to all names in the list.  If a
    str, 

In [22]:
widgets.interactive.update?

Signature: widgets.interactive.update(self, *args)
Docstring:
Call the interact function and update the output widget with
the result of the function call.

Parameters
----------
*args : ignored
    Required for this method to be used as traitlets callback.
File:      ~/opt/anaconda3/envs/covid_viz_env/lib/python3.8/site-packages/ipywidgets/widgets/interaction.py
Type:      function


In [ ]:
widgets.interactive_output.